#Setup

In [ ]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

# Running the IndicNER Model

Let's try annotating some Indian language sentences and get the named entities

In [ ]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
# # let us try with some example sentences here
# sentence='ఎన్నికల ప్రకటన వెలువడినది లగాయతు.. సీట్ల పంపకం కోసం కొట్టుకోవడమే మహాకూటమికి సరిపోతున్నది. ప్రతిరోజూ ఉదయం ఇంట్లో టిఫిన్ చెయ్యడం, గోల్కొండ రిసారట్స్‌కు వెళ్లి పిచ్చాపాటీ మాట్లాడుకుని భోజనం చెయ్యడం, ఇంటికి వెళ్లడం, మరునాడు మళ్ళీ అదే తంతు…కాంగ్రెస్ పార్టీకి కార్యకర్తలు తక్కువ, నాయకులు ఎక్కువ. సర్పంచ్ గా కూడా చేయనివాడు ముఖ్యమంత్రి కావాలనుకుంటారు ఆ పార్టీలో. అందుకే ఎప్పుడు ఎన్నికలు వచ్చినా, కాంగ్రెస్ పార్టీలో అంతర్గత యుద్ధం మొదలవుతుంది. వారికి వారే వెన్నుపోట్లు పొడుచుకుంటారు. తమకు ముప్ఫయి సీట్లు కావాలని టీజేఎస్, తమకు నలభై కావాలని తెలుగుదేశం కాంగ్రెస్ పార్టీకి హెచ్చరికలు జారీ చేస్తున్నాయి. లేకపోతె తమ దారి తాము చూసుకుంటామని బెదిరింపులకు దిగుతున్నాయి. ఆ రెండు పార్టీలకే అరవై డ్బ్బై స్థానాలు ఇచ్చి ఇక కాంగ్రెస్ పోటీ చేసేది ఎన్ని స్థానాలలో??? నవ్వు రావడం లేదూ? ఎన్నికలు అయ్యేంతవరకూ పరస్పరం కలహించుకోవడంలోనే కాలక్షేపం చేసే మహాకూటమి ఎన్నికల్లో పోటీ చేసి విజయం సాధించడం అయ్యేపనేనా?'
# predicted_labels = get_predictions(sentence=sentence,
#                                    tokenizer=tokenizer,
#                                    model=model
#                                    )

# for index in range(len(sentence.split(' '))):
#   print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

ఎన్నికల	O
ప్రకటన	O
వెలువడినది	O
లగాయతు..	O
సీట్ల	O
పంపకం	O
కోసం	O
కొట్టుకోవడమే	O
మహాకూటమికి	O
సరిపోతున్నది.	O
ప్రతిరోజూ	O
ఉదయం	O
ఇంట్లో	O
టిఫిన్	O
చెయ్యడం,	O
గోల్కొండ	O
రిసారట్స్‌కు	O
వెళ్లి	O
పిచ్చాపాటీ	O
మాట్లాడుకుని	O
భోజనం	O
చెయ్యడం,	O
ఇంటికి	O
వెళ్లడం,	O
మరునాడు	O
మళ్ళీ	O
అదే	O
తంతు…కాంగ్రెస్	O
పార్టీకి	O
కార్యకర్తలు	O
తక్కువ,	O
నాయకులు	O
ఎక్కువ.	O
సర్పంచ్	O
గా	O
కూడా	B-ORG
చేయనివాడు	O
ముఖ్యమంత్రి	O
కావాలనుకుంటారు	O
ఆ	O
పార్టీలో.	O
అందుకే	O
ఎప్పుడు	O
ఎన్నికలు	O
వచ్చినా,	O
కాంగ్రెస్	O
పార్టీలో	O
అంతర్గత	O
యుద్ధం	O
మొదలవుతుంది.	O
వారికి	O
వారే	O
వెన్నుపోట్లు	O
పొడుచుకుంటారు.	O
తమకు	O
ముప్ఫయి	O
సీట్లు	O
కావాలని	B-ORG
టీజేఎస్,	O
తమకు	O
నలభై	O
కావాలని	O
తెలుగుదేశం	O
కాంగ్రెస్	O
పార్టీకి	O
హెచ్చరికలు	O
జారీ	O
చేస్తున్నాయి.	O
లేకపోతె	O
తమ	O
దారి	O
తాము	O
చూసుకుంటామని	B-ORG
బెదిరింపులకు	O
దిగుతున్నాయి.	O
ఆ	O
రెండు	O
పార్టీలకే	B-ORG
అరవై	I-ORG
డ్బ్బై	O
స్థానాలు	O
ఇచ్చి	O
ఇక	O
కాంగ్రెస్	O
పోటీ	O
చేసేది	O
ఎన్ని	O
స్థానాలలో???	O
నవ్వు	O
రావడం	O
లేదూ?	O
ఎన్నికలు	O
అయ్యేంతవరకూ	O
పరస్పరం	O
కలహిం

#Naampadam Dataset

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

In [ ]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='te'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

In [ ]:
small_train_dataset = raw_datasets["train"].select(range(20000))

# Print the length of the small dataset
print(len(small_train_dataset))
small_train_dataset.column_names

20000


['tokens', 'ner_tags']

In [ ]:
# let's print an instance of dataset
idx=1
rec=small_train_dataset[idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))


ఈ	0
స్మార్ట్	0
ఫోన్	0
ఇప్పటికే	0
ఇండోనేషియాలో	5
లాంచ్	0
అయింది	0
.	0


In [ ]:
column_names = small_train_dataset.column_names
print(column_names)

features = small_train_dataset.features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [ ]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [ ]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


# Training an NER Model with the dataset

We have already seen how to get predictions from fine-tuned NER model. We will now use the pre-trained IndicBERT model and fine-tune it for NER task.

**Let** us download a pre-trained model and fine-tune it for the task of NER. We will have to use the `AutoModelForTokenClassification` class to fine-tune the model

**Load Pre-trained Model**

In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

# Define the number of epochs
num_epochs = 3

# Load configuration, tokenizer, and model
config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )



Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

#model=model.to("cuda")
!pip install --upgrade protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


**Tokenize all texts and align the labels with them**

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset = small_train_dataset
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=6,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=6):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=6,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

**Create Data Collator, Metrics**

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


**Set Training Arguments**

In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.6 MB/s eta 0:00:00



**Training**

In [ ]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True
# Define training arguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,  # Set the number of epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy='steps',
    eval_steps=100,
    logging_dir='./logs',
    logging_steps=10,
    disable_tqdm=False,
    load_best_model_at_end=True,  # Set load_best_model_at_end to True
    learning_rate=0.0001,  # Set the learning rate here
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)




/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=100,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

Step,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
100,0.278900,0.389858,0.710586,0.604406,0.653209,1044,0.493237,0.527483,0.509786,1037,0.686427,0.618263,0.650564,2004,0.635718,0.591677,0.612907,0.877775
200,0.419700,0.374921,0.642646,0.623563,0.632961,1044,0.546354,0.426230,0.478873,1037,0.652196,0.652196,0.652196,2004,0.627287,0.587515,0.606750,0.878929
300,0.305700,0.380523,0.706208,0.610153,0.654676,1044,0.533199,0.511090,0.521910,1037,0.624081,0.677645,0.649761,2004,0.620088,0.618115,0.619100,0.880587
400,0.282700,0.398674,0.703867,0.610153,0.653669,1044,0.604255,0.410800,0.489093,1037,0.689694,0.617764,0.651750,2004,0.675771,0.563280,0.614419,0.879686
500,0.377000,0.373684,0.657063,0.677203,0.666981,1044,0.497854,0.559306,0.526794,1037,0.641812,0.678643,0.659714,2004,0.607110,0.647980,0.626880,0.876225
600,0.361300,0.363807,0.689725,0.649425,0.668969,1044,0.509044,0.569913,0.537762,1037,0.693803,0.659182,0.676049,2004,0.639822,0.634027,0.636911,0.886029
700,0.321200,0.376251,0.683787,0.650383,0.666667,1044,0.556624,0.502411,0.528130,1037,0.703725,0.622255,0.660487,2004,0.661173,0.599021,0.628564,0.883651
800,0.272900,0.387210,0.677483,0.659962,0.668607,1044,0.619910,0.396336,0.483529,1037,0.678820,0.631737,0.654433,2004,0.667419,0.579192,0.620183,0.879253
900,0.316300,0.364715,0.629787,0.708812,0.666967,1044,0.598361,0.492768,0.540455,1037,0.645785,0.699601,0.671617,2004,0.631667,0.649449,0.640435,0.884191
1000,0.315000,0.360287,0.750000,0.603448,0.668790,1044,0.579268,0.458052,0.511578,1037,0.656652,0.687126,0.671544,2004,0.660633,0.607589,0.633002,0.885561


Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [ ]:

metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       2.24
  eval_LOC_f1             =     0.6773
  eval_LOC_number         =       1044
  eval_LOC_precision      =     0.7077
  eval_LOC_recall         =     0.6494
  eval_ORG_f1             =     0.5598
  eval_ORG_number         =       1037
  eval_ORG_precision      =     0.6038
  eval_ORG_recall         =     0.5217
  eval_PER_f1             =     0.6936
  eval_PER_number         =       2004
  eval_PER_precision      =     0.6859
  eval_PER_recall         =     0.7016
  eval_loss               =     0.3483
  eval_overall_accuracy   =     0.8941
  eval_overall_f1         =     0.6572
  eval_overall_precision  =     0.6724
  eval_overall_recall     =     0.6426
  eval_runtime            = 0:00:31.07
  eval_samples_per_second =     86.889
  eval_steps_per_second   =      5.439


In [ ]:
test_dataset = raw_datasets['test']
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=6,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [ ]:

# Evaluate on the test dataset
metrics = trainer.evaluate(test_dataset)

trainer.log_metrics("test", metrics)

***** test metrics *****
  epoch                   =       2.24
  eval_LOC_f1             =     0.7075
  eval_LOC_number         =        483
  eval_LOC_precision      =      0.801
  eval_LOC_recall         =     0.6335
  eval_ORG_f1             =     0.6178
  eval_ORG_number         =        263
  eval_ORG_precision      =     0.6275
  eval_ORG_recall         =     0.6084
  eval_PER_f1             =     0.7644
  eval_PER_number         =        609
  eval_PER_precision      =     0.7753
  eval_PER_recall         =     0.7537
  eval_loss               =     0.2862
  eval_overall_accuracy   =     0.9134
  eval_overall_f1         =     0.7159
  eval_overall_precision  =     0.7526
  eval_overall_recall     =     0.6827
  eval_runtime            = 0:00:09.74
  eval_samples_per_second =     86.883
  eval_steps_per_second   =      5.437


In [ ]:
print(test_dataset)

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 847
})


##Question 4:

In [ ]:
data = [
    {
        "tokens": [
            "అయితే", "ఈ", "సారి", "అనూహ్యంగా", "కోటి", "యాభై", "లక్షల",
            "పలకడంతో", "ఆ", "డబ్బును", "ఏం", "చేయాలో", "ఇంకా", "నిర్ణయించుకోలేదని",
            "గోయట్", "చెప్పుకొచ్చాడు"
        ],
        "ner_tags": [
            0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
        ]
    }
]


In [ ]:
# Assuming your function expects "text" for the text data and "labels" for the labels
# For testing, you might not need actual labels, but let's include them for the sake of format

print(data[0])

{'tokens': ['అయితే', 'ఈ', 'సారి', 'అనూహ్యంగా', 'కోటి', 'యాభై', 'లక్షల', 'పలకడంతో', 'ఆ', 'డబ్బును', 'ఏం', 'చేయాలో', 'ఇంకా', 'నిర్ణయించుకోలేదని', 'గోయట్', 'చెప్పుకొచ్చాడు'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]}


In [ ]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence

# Assuming your data is structured like this:
data = [
{   "tokems": [     "అయితే",     "ఈ",     "సారి",     "అనూహ్యంగా",     "కోటి",     "యాభై",     "లక్షల",     "పలకడంతో",     "ఆ",     "డబ్బును",     "ఏం",     "చేయాలో",     "ఇంకా",     "నిర్ణయించుకోలేదని",     "గోయట్",     "చెప్పుకొచ్చాడు"   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "B-PER",     "O",     "O"   ] },
{   "tokems": [     "ప్రజల",     "మనోభావాలు,",     "ఎమోషన్స్",     "కి",     "అంతగా",     "ప్రాముఖ్యత",     "ఇవ్వకుండా",     "తానుఅనుకున్న",     "పనులు",     "చక్కబెట్టేవారు",   " ."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",  "O"   ] },
{   "tokems": [     "అప్పుడు,",     "మీరు",     "చివరకు",     "చూస్తున్న",     "పెరుగుదల",     "అలవాటు",     "రకం",     "ప్రోత్సహించడానికి",     "న్యాయమైన",     "కట్స్",     "తయారు",  "."  ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O", "O"   ] },
{   "tokems": [ "ఎన్నికల", "ప్రకటన", "వెలువడినది", "లగాయతు", "సీట్ల", "పంపకం", "కోసం", "కొట్టుకోవడమే", "మహాకూటమికి", "సరిపోతున్నది", "ప్రతిరోజూ", "ఉదయం", "ఇంట్లో", "టిఫిన్", "చెయ్యడం", "గోల్కొండ", "రిసారట్స్కు", "వెళ్లి", "పిచ్చాపాటీ", "మాట్లాడుకుని", "భోజనం", "చెయ్యడం", "ఇంటికి", "వెళ్లడం", "మరునాడు", "మళ్ళీ", "అదే", "తంతు", "కాంగ్రెస్", "పార్టీకి", "కార్యకర్తలు", "తక్కువ", "నాయకులు", "ఎక్కువ", "సర్పంచ్", "గా", "కూడా", "చేయనివాడు", "ముఖ్యమంత్రి", "కావాలనుకుంటారు", "ఆ", "పార్టీలో", "అందుకే", "ఎప్పుడు", "ఎన్నికలు", "వచ్చినా", "కాంగ్రెస్", "పార్టీలో", "అంతర్గత", "యుద్ధం", "మొదలవుతుంది", "వారికి", "వారే", "వెన్నుపోట్లు", "పొడుచుకుంటారు", "తమకు", "ముప్ఫయి", "సీట్లు", "కావాలని", "టీజేఎస్", "తమకు", "నలభై", "కావాలని", "తెలుగుదేశం", "కాంగ్రెస్", "పార్టీకి", "హెచ్చరికలు", "జారీ", "చేస్తున్నాయి", "లేకపోతె", "తమ", "దారి", "తాము", "చూసుకుంటామని", "బెదిరింపులకు", "దిగుతున్నాయి", "ఆ", "రెండు", "పార్టీలకే", "అరవై", "డ్బ్బై", "స్థానాలు", "ఇచ్చి", "ఇక", "కాంగ్రెస్", "పోటీ", "చేసేది", "ఎన్ని", "స్థానాలలో", "నవ్వు", "రావడం", "లేదూ", "ఎన్నికలు", "అయ్యేంతవరకూ", "పరస్పరం", "కలహించుకోవడంలోనే", "కాలక్షేపం", "మహాకూటమి", "ఎన్నికల్లో", "పోటీ", "చేసి", "విజయం", "సాధించడం", "అయ్యేపనేనా" ], "ner_tags": [ "O", "O", "O", "O", "B-LOC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "I-LOC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "B-ORG", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "O", "B-ORG", "I-ORG", "O", "O", "O", "O", "O", "O", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
{ "tokems": [ "మెగా", "పవర్", "స్టార్", "రామ్", "చరణ్", ",", "ఊరమాస్", "డైరెక్టర్", "బోయపాటి", "శ్రీను", "కాంబినేషన్‌లో", ",", "ఫ్యామిలీ", ",", "లవ్", "అండ్", "యాక్షన్", "ఎంటర్‌టైనర్‌గా", "రూపొందిన", "వినయ", "విధేయ", "రామ్", ",", "జనవరి", "11న", "వరల్డ్", "వైడ్", "గ్రాండ్", "రిలీజ్‌కి", "రెఢీ", "అయిపోయింది", ".", "డి.వి.వి.", "దానయ్య", "నిర్మించగా", ",", "కైరా", "అద్వాణీ", "హీరోయిన్‌గా", "నటించింది" ], "ner_tags": [ "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "I-PER", "O", "O" ] },
{ "tokems": [ "అది,", "పొందుటకు", ",",  "మేము", "ప్రధాన", "ద్వారం", "నుండి", "కుడి", "చెయ్యి", "మరియు", "నిరంతరం", "పెరుగుతుంది", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
  { "tokems": [ "లోతైన", "సమావేశంలో", ",", "సారాంశం", "మానవ", "శరీరంలో", "నుండి", "వేరు", ",", "అస్తిత్వ", "మరణం", "అని", "పిలువబడుతుంది", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 { "tokems": [ "చాటుసాహిత్యంలో", "తరచూ", "కనిపించే", "లక్షణం", "–", "పద్యాల", "గురించీ", ",", "కవుల", "గురించీ", "జనవ్యవహారంలో", "వ్యాప్తమైయ్యే", "కథలు", "." ], "ner_tags": [ "O", "O", "O",  "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
{ "tokems": [ "బ్రేకింగ్", ":", "మళ్ళీ", "వైసీపీలోకి", "ఎంట్రీ", "ఇస్తున్న", "కీలక", "నేత", "..", "షాక్‌లో", "టీడీపీ", "." ], "ner_tags": [ "O", "O", "O", "B-ORG", "O", "O", "O", "O", "O", "O", "B-ORG", "O"] },
 { "tokems": [ "ప్రామాణిక", "బ్యాంకు", "రుణ", "జారీ", "కాకుండా", ",", "క్రెడిట్", "యొక్క", "లైన్", "ఆర్థిక", "సంస్థల", "రుణగ్రహీతలు", "మరింత", "ఆకర్షణీయంగా", ",", "అలాగే", "ఉంది", "." ], "ner_tags": [ "O", "B-ORG", "O", "O", "O", "O", "O", "O", "O", "B-ORG", "I-ORG", "O", "O", "O", "O", "O", "O", "O" ] },
 { "tokems": [ "వారి", "ప్రకాశవంతమైన", "నక్షత్రాలు", "సిరియస్", "మరియు", "Procyon", "ఏ", "ఇతర", "తో", "తికమక", "కష్టం", "." ], "ner_tags": [ "O", "O", "O", "B-PER", "O", "B-PER", "O", "O", "O", "O", "O", "O" ] },
 { "tokems": [ "పీటర్స్బర్గ్", "పతనం", "తరువాత", "లీ", "యొక్క", "వెనుకవైపు", "ఉన్న", "సైనికదళాన్ని", "కొనసాగించడంతో", ",",  "రైట్", "మరియు", "VI", "కార్ప్స్", "మళ్లీ", "షెరిడాన్", "దిశగా", "వచ్చారు", "." ], "ner_tags": [ "B-PER", "O", "O", "B-PER", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "B-PER",  "O", "B-LOC", "O", "O", "O" ] },
 { "tokems": [ "దానిపై", "హైడ్", "ప్రదర్శన", "లక్షణాలు", "ఒక", "ప్రాథమిక", "వివరణ", "తో", "ప్రారంభం", "కావాలి", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
{ "tokems": [ "మోరెనా", "జిల్లా", "జౌరా", "ప్రాంతంలో", "జరిగిన", "ఒక", "ఎన్నికల", "సభలో", "రాహుల్", "మాట్లాడుతూ", "ఎంజే", "అక్బర్‌పై", "పలువురు", "మహిళా", "జర్నలిస్టులు", "చేసిన", "లైంగిక", "దాడి", "ఆరోపణలను", "దృష్టిలో", "పెట్టుకొని", "మోదీ", "ప్రభుత్వాన్ని", "నిలదీశారు", "." ], "ner_tags": [ "B-LOC", "O", "B-LOC", "O", "O", "O", "O", "O", "B-PER", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O" ] },

 #  { "tokems": [ "కాన్స్టాంటినోపుల్లో", "బోహెంండ్", "ముఖ్యంగా", "స్వాగతం", "లేదు", "ఎందుకంటే", "అతని", "తండ్రి", ",", "రాబర్ట్", "గైకార్డ్", ",", "బైజాంటైన్", "సామ్రాజ్యాన్ని", "ఆక్రమించి", ",", "డియర్హచ్యూమ్", "మరియు", "కోర్ఫు", "నగరాలను", "స్వాధీనం", "చేసుకున్నాడు", "." ], "ner_tags": [ 6, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 6, 0, 0, 0, 6, 0, 6, 0, 0, 0, 0 ] },
 #   { "tokems": [ "శిఖరం", "(", "ఉత్తమ", "సమయం", "-", "సాయంత్రం", "ముఖ్యంగా", "“", "వర్షం", "కింద", "“", ")", "యొక్క", "వేడి", "మానుకోండి", ";" ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 #  { "tokems": [ "ఏదైనా", "ఒక", "నీటి", "పారుదల", "ప్రాజెక్టుకు", "జాతీయ", "హోదాను", "ఇవ్వాలని", "కోరే", "అవకాశం", "ఉంది", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 #{ "tokems": [ "ప్రైవేట్స్‌", "పోల్స్‌ను", "బట్టి", "చూస్తే", "వితిక", "పరిస్థితి", "మాత్రం", "దారుణంగా", "ఉన్నట్లు", "తెలుస్తోంది", "." ], "ner_tags": [ 0,0,0,0,0,0,0,0,0,0,0 ] },
 # { "tokems": [ "బంగ్లా", "ఓపెనర్", "షాద్‌మన్‌", "ఇస్లామ్‌.", "(", "29", ")", "ఫర్వాలేదనిపించగా", "...", "మరో", "ఓపెనర్", "ఇమ్రుల్‌", "(", "4", ")", "నిరాశపరిచాడు", "." ], "ner_tags": [ "B-LOC", "O", "B-PER", "I-PER", "O", "O", "O", "O", "O", "O", "O", "O", "B-PER", "O", "O", "O", "O" ] },
 # { "tokems": [ "అయితే", "వాటికి", "హైనెక్‌", "బ్లవుజులు", ",", "కాలర్స్‌", ",", "ఫుల్‌స్లీవ్స్‌", ",", "క్లోజ్డ్‌", ",", "బోట్‌", ",", "బ్యాక్‌హైనెక్", "‌,", "ఫ్రంట్‌", "రౌండ్‌నెక్‌", "బ్లవుజ్‌లు", "బాగుంటాయి", "." ], "ner_tags": [ "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O" ] },
 # {    "tokems": [     "4",     ")",     "పైన",     "చెప్పిన",     "వి",     "అనగా",     "కలిపి",     "పెట్టుకున్న",     "రవ్వ",     ",",     "ఉడికించిన",     "ఆలూ",     ",",     "కట్",     "చెసి",     "పెట్టకున్నవి",     "అన్ని",     "ఇంకా",     "జిలకర్ర",     ",",     "ఉప్పు",     "రుచి",     "కి",     "సరిపడ",     ".",     "వెసి",     "ముద్ద",     "లా",     "కలుపుకోవాలీ",     "."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O"   ] },
 #{   "tokems": [     "సినిమాలో",     "కంటెంట్",     "అంతంతమాత్రమే",     "అయినప్పటికీ",     "..",     "ఈ",     "మాత్రం",     "వసూళ్లు",     "వచ్చాయంటే",     "అది",     "ఎన్టీఆర్",     "పెర్ఫామెన్స్‌",     "వల్లే",     "."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "O",     "B-PER",     "O",     "O"   ] },
  #{   "tokems": [     "‘",     "స్టార్స్",     "‌’",     "(",     "స్కీమ్‌",     "ఫర్‌",     "ట్రాన్స్‌ఫర్మేషన్‌",     "అండ్‌",     "అడ్వాన్స్‌డ్‌",     "రీసెర్చ్‌",     "ఇన్‌",     "బేసిక్‌",     "సైన్సెస్",     ")",     ",",     "ఐఐఎస్‌సీ",     "బెంగళూరు",     "ద్వారా",     "అధ్యయన",     "కార్యక్రమాలను",     "మంజూరు",     "చేస్తున్నారు",     "."   ],   "ner_tags": [     "O",     "O",     "O",     "O",     "B-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "I-ORG",     "O",     "O",     "B-ORG",     "B-LOC",     "O",     "O",     "O",     "O",     "O",     "O",     "O"   ] }
    # More records...
]

# Transform your list of dictionaries into the expected format
transformed_data = {
    "tokems": [item["tokems"] for item in data],
    "ner_tags": [item["ner_tags"] for item in data],
}

# Define the features of your dataset, aligning with your manual data structure
features = Features({
    'tokems': Sequence(feature=Value(dtype='string')),
    'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))
})

# Create a Dataset object from your transformed data
question1_data = Dataset.from_dict(transformed_data, features=features)

# Now you can access column_names and features just like your training dataset
column_names = question1_data.column_names
print(column_names)

dataset_features = question1_data.features
print(dataset_features)


['tokems', 'ner_tags']
{'tokems': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [ ]:
print(question1_data)

Dataset({
    features: ['tokems', 'ner_tags'],
    num_rows: 23
})


In [ ]:
test_dataset = question1_data
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=6,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=6):   0%|          | 0/14 [00:00<?, ? examples/s]

In [ ]:

# Evaluate on the test dataset
metrics = trainer.evaluate(test_dataset)

trainer.log_metrics("test", metrics)

***** test metrics *****
  epoch                   =       2.24
  eval_LOC_f1             =     0.5714
  eval_LOC_number         =          5
  eval_LOC_precision      =        1.0
  eval_LOC_recall         =        0.4
  eval_ORG_f1             =     0.1429
  eval_ORG_number         =         11
  eval_ORG_precision      =     0.3333
  eval_ORG_recall         =     0.0909
  eval_PER_f1             =     0.4545
  eval_PER_number         =         14
  eval_PER_precision      =      0.625
  eval_PER_recall         =     0.3571
  eval_loss               =     0.4211
  eval_overall_accuracy   =     0.9128
  eval_overall_f1         =     0.3721
  eval_overall_precision  =     0.6154
  eval_overall_recall     =     0.2667
  eval_runtime            = 0:00:00.20
  eval_samples_per_second =     68.868
  eval_steps_per_second   =      4.919
